In [31]:
from nltk.stem import PorterStemmer

In [1]:
!ls -ltrah /workspace/datasets

total 378M
drwxr-xr-x  5 gitpod gitpod   55 Aug 15  2012 product_data
-rw-r--r--  1 gitpod gitpod 1.7K Dec 11  2019 popular_skus.py
-rw-r--r--  1 gitpod gitpod 146M Dec 11  2019 test.csv
-rw-r--r--  1 gitpod gitpod 233M Dec 11  2019 train.csv
drwxr-xr-x  4 gitpod gitpod   98 Jun 28 11:47 .
drwxr-xr-x  2 gitpod gitpod 4.0K Jul  1 13:12 fasttext
drwxr-xr-x 14 gitpod gitpod 4.0K Jul  5 10:44 ..


In [2]:
import pandas as pd

# Category DataFrame

In [2]:
import xml.etree.ElementTree as ET

cat_file = '/workspace/datasets/product_data/categories/categories_0001_abcat0010000_to_pcmcat99300050000.xml'
tree = ET.parse(cat_file)
root = tree.getroot()

In [3]:
subcats = []

In [4]:
for category in root:
    category_id = category.find('id').text
    subcategories = category.find('subCategories')
    for subcategory in subcategories:
        subcat_id = subcategory.find('id').text
        subcats.append([subcat_id, category_id])

In [5]:
sdf = pd.DataFrame(subcats, columns=['category', 'parent'])

In [7]:
sdf.head()

,category,parent
0,pcmcat140000050035,abcat0010000
1,pcmcat140000050036,abcat0010000
2,pcmcat140000050037,abcat0010000
3,pcmcat140000050039,abcat0010000
4,pcmcat140000050040,abcat0010000


In [8]:
sdf[sdf['category'].duplicated(keep=False)].sort_values('category')

,category,parent
34,abcat0014001,abcat0014000
41,abcat0014001,abcat0020000
164,abcat0101008,abcat0107001
64,abcat0101008,abcat0101000
464,abcat0102008,abcat0303000
...,...,...
412,pcmcat59600050005,abcat0302000
440,pcmcat59600050005,abcat0302019
472,pcmcat59600050005,abcat0304000
1674,pcmcat59600050005,cat00000


In [9]:
sdf.drop_duplicates(subset=['category'], keep='first', inplace=True, ignore_index=True)

In [10]:
sdf = sdf.sort_values('category').reset_index(drop=True)

In [11]:
sdf

,category,parent
0,abcat0010000,cat00000
1,abcat0011000,abcat0010000
2,abcat0011001,abcat0011000
3,abcat0011002,abcat0011000
4,abcat0011003,abcat0011000
...,...,...
4634,pcmcat97200050013,cat15205
4635,pcmcat97200050015,cat15063
4636,pcmcat99000050001,pcmcat50000050006
4637,pcmcat99000050002,pcmcat99000050001


In [12]:
len(sdf)

4639

# Product

In [13]:
all_clicks = '/workspace/datasets/train.csv'

In [14]:
df = pd.read_csv(all_clicks)

In [15]:
df.head()

,user,sku,category,query,click_time,query_time
0,000000df17cd56a5df4a94074e133c9d4739fae3,2125233,abcat0101001,Televisiones Panasonic 50 pulgadas,2011-09-01 23:44:52.533,2011-09-01 23:43:59.752
1,000001928162247ffaf63185cd8b2a244c78e7c6,2009324,abcat0101001,Sharp,2011-09-05 12:25:37.42,2011-09-05 12:25:01.187
2,000017f79c2b5da56721f22f9fdd726b13daf8e8,1517163,pcmcat193100050014,nook,2011-08-24 12:56:58.91,2011-08-24 12:55:13.012
3,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877125,abcat0101001,rca,2011-10-25 07:18:14.722,2011-10-25 07:16:51.759
4,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877134,abcat0101005,rca,2011-10-25 07:19:51.697,2011-10-25 07:16:51.759


In [16]:
df['category_exists'] = df['category'].isin(sdf['category'])

In [36]:
df_valid = df[df['category_exists']]
df_valid = df_valid[['category', 'query']].copy().reset_index(drop=True)

In [38]:
df_valid['query_normalized'] = df_valid['query'].str.strip().str.lower()

In [39]:
df_valid['query_normalized'] = df_valid['query_normalized'].str.replace(r"[^a-zA-Z0-9\s]", " ", regex=True)

In [40]:
stemmer = PorterStemmer()

In [43]:
df_valid['query_normalized'] = df_valid.apply(lambda x: stemmer.stem(x['query_normalized']), axis=1)

In [44]:
df_valid['count'] = 1

In [45]:
df_valid.head(10)

,category,query,query_normalized,count
0,abcat0101001,Televisiones Panasonic 50 pulgadas,televisiones panasonic 50 pulgada,1
1,abcat0101001,Sharp,sharp,1
2,pcmcat193100050014,nook,nook,1
3,abcat0101001,rca,rca,1
4,abcat0101005,rca,rca,1
5,pcmcat143200050016,Flat screen tvs,flat screen tv,1
6,pcmcat247400050001,macbook,macbook,1
7,pcmcat171900050028,Blue tooth headphones,blue tooth headphon,1
8,abcat0107004,Tv antenna,tv antenna,1
9,pcmcat186100050006,memory card,memory card,1


In [69]:
df_querycount = df_valid

In [47]:
sdf_lookup = sdf.set_index('category').sort_index()

In [65]:
sdf_lookup = parents_df.set_index('category').sort_index()

In [66]:
complete = False

In [67]:
threshold = 1000

In [68]:
root_category = 'cat00000'

In [70]:
def find_parent(r):
    if r['count'] >= threshold:
        return r['category']
    if r['category'] == root_category:
        return r['category']
    parent = sdf_lookup.loc[r['category']]['parent']
    if parent == root_category:
        return r['category']
    return parent    

In [71]:
while not complete:
    df_querycount = df_querycount.groupby(['category', 'query_normalized'], as_index=False)['count'].sum()
    print('total rows: ', len(df_querycount))
    print('unique cats: ', len(df_querycount['category'].unique()))
    new_categories = df_querycount.apply(find_parent, axis=1)
    complete = df_querycount['category'].equals(new_categories)
    df_querycount['category'] = new_categories

total rows:  326925
unique cats:  1486
total rows:  296294
unique cats:  414
total rows:  260205
unique cats:  177
total rows:  235156
unique cats:  111
total rows:  229543
unique cats:  93
total rows:  228307
unique cats:  89
total rows:  227916
unique cats:  89


In [54]:
tree = ET.parse(cat_file)
root = tree.getroot()

In [57]:

# Parse the category XML file to map each category id to its parent category id in a dataframe.
categories = []
parents = []
for child in root:
    id = child.find('id').text
    cat_path = child.find('path')
    cat_path_ids = [cat.find('id').text for cat in cat_path]
    leaf_id = cat_path_ids[-1]
    if leaf_id != root_category:
        categories.append(leaf_id)
        parents.append(cat_path_ids[-2])
parents_df = pd.DataFrame(list(zip(categories, parents)), columns =['category', 'parent'])


In [58]:
parents_df

,category,parent
0,abcat0010000,cat00000
1,abcat0011000,abcat0010000
2,abcat0011001,abcat0011000
3,abcat0011002,abcat0011000
4,abcat0011003,abcat0011000
...,...,...
4634,pcmcat97200050013,cat15205
4635,pcmcat97200050015,cat15063
4636,pcmcat99000050001,pcmcat50000050006
4637,pcmcat99000050002,pcmcat99000050001


In [59]:
sdf

,category,parent
0,abcat0010000,cat00000
1,abcat0011000,abcat0010000
2,abcat0011001,abcat0011000
3,abcat0011002,abcat0011000
4,abcat0011003,abcat0011000
...,...,...
4634,pcmcat97200050013,cat15205
4635,pcmcat97200050015,cat15063
4636,pcmcat99000050001,pcmcat50000050006
4637,pcmcat99000050002,pcmcat99000050001


In [64]:
parents_df.set_index('category').sort_index().compare(sdf.set_index('category').sort_index())

parent                    
                                  self               other
category                                                  
abcat0106002        pcmcat277400050005        abcat0106000
abcat0106007        pcmcat277400050005        abcat0106003
abcat0106008        pcmcat277400050005        abcat0106000
abcat0106010        pcmcat253300050003        abcat0106000
abcat0106019              abcat0913000        abcat0106010
...                                ...                 ...
pcmcat275800050015  pcmcat243400050029  pcmcat153600050007
pcmcat275800050016  pcmcat243400050029  pcmcat153600050007
pcmcat278100050001  pcmcat232900050000  pcmcat153600050007
pcmcat278100050002  pcmcat232900050000  pcmcat153600050007
pcmcat59600050005         abcat0302000        abcat0300000

[204 rows x 2 columns]

In [72]:
df_querycount

,category,query_normalized,count
0,abcat0010000,5774879,1
1,abcat0010000,belkin n600,1
2,abcat0010000,beverage cent,1
3,abcat0010000,cell phon,1
4,abcat0010000,cellphon,1
...,...,...,...
227911,pcmcat253300050001,transformers toy,1
227912,pcmcat253300050001,truck accessori,1
227913,pcmcat253300050001,white ipod touch,2
227914,pcmcat253300050001,wireless camera,2


In [61]:
parents_df.compare(sdf)

parent                    
                    self               other
54    pcmcat277400050005        abcat0106000
59    pcmcat277400050005        abcat0106003
60    pcmcat277400050005        abcat0106000
62    pcmcat253300050003        abcat0106000
71          abcat0913000        abcat0106010
...                  ...                 ...
4537  pcmcat243400050029  pcmcat153600050007
4538  pcmcat243400050029  pcmcat153600050007
4557  pcmcat232900050000  pcmcat153600050007
4558  pcmcat232900050000  pcmcat153600050007
4586        abcat0302000        abcat0300000

[204 rows x 2 columns]

In [55]:
!head -100 $cat_file

<categories>
  <category>
    <id>abcat0010000</id>
    <name>Gift Center</name>
    <path>
      <category>
        <id>cat00000</id>
        <name>Best Buy</name>
      </category>
      <category>
        <id>abcat0010000</id>
        <name>Gift Center</name>
      </category>
    </path>
    <subCategories>
      <category>
        <id>pcmcat140000050035</id>
        <name>Capturing Photos &amp; Videos</name>
      </category>
      <category>
        <id>pcmcat140000050036</id>
        <name>Listening to Digital Music</name>
      </category>
      <category>
        <id>pcmcat140000050037</id>
        <name>Computing Made Easy</name>
      </category>
      <category>
        <id>pcmcat140000050039</id>
        <name>Simple GPS Navigation</name>
      </category>
      <category>
        <id>pcmcat140000050040</id>
        <name>Playing Video Games</name>
      </category>
      <category>
        <id>pcmcat140000050041</id>
        <name>Watching HDTV</name>
      </category>
  

In [6]:
sdf.head()

,category,parent
0,pcmcat140000050035,abcat0010000
1,pcmcat140000050036,abcat0010000
2,pcmcat140000050037,abcat0010000
3,pcmcat140000050039,abcat0010000
4,pcmcat140000050040,abcat0010000


In [7]:
sdf[sdf['category'] == 'abcat0600000']

,category,parent
1667,abcat0600000,cat00000
